# Data Aggregation for the "Interpreting Emotion in Messages" Study

* Project and code written by Erik Jacobson

<hr>

## Getting the data from an API

* The data is stored in a SQL database through the cloud, and can be accessed using an API.

* The API is only accessible if you have a key, which itself can only be obtained through administrator accounts.

To access the data, I need to make a request to the API endpoint and store the returned JSON data in a file.

In [1]:
##
# Getting the Data
#
import pandas as pd
import numpy as np
import requests
import json

# The data can be accessed through an API
studySite = "https://emotion.ewjresearch.com/"
apiKey = open("APIKey.txt", "r").read()

headers = {'Authorization' : apiKey, 'Accept': 'application/json'}

# Make the request to get the users resource
response = requests.get(studySite + "api/users", headers=headers)

if(response.status_code != 200): # If not successful
    print(response.status_code)
    print(response.text)
    print('The server didn\'t respond. Using backup JSON data for now.')
    print('Get a new key to access the new data.')
else: # If successful
    # Write the JSON to a data file
    with open("UserJson.json", "w") as dataFile:
        # Get what's inside of data array
        data = json.loads(response.text)['data']
        dataFile.write(json.dumps(data))
        dataFile.close()
    print("Successfully downloaded User data from " + studySite)

Successfully downloaded User data from https://emotion.ewjresearch.com/


All the JSON data is written to a file which we'll call `UserJson.json`. This way, I can manipulate the raw data without having to request the server each time.

<hr>
    
## Creating a Users Dataframe

- The data is formatted in JSON with users containing an array `records` containing individual objects that represent each of the questions on the app.

- Each user object should be represented as a row in the excel output, with attributes (such as `username`) represented as a column.

In [2]:
##
# Loading the data from the JSON
#
import json
import pandas as pd
from pandas.io.json import json_normalize

# Setup for excel writing
excel = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')

# Getting the initial user information
data = json.loads(open("UserJson.json", "r").read())
columns =([
    'id',
    'username',
    'computer',
    'researcher',
    'admin',
    'overtime',
    'credit_granted',
    'progress',
    'age',
    'primary_language',
    'facebook_use',
    'instagram_use',
    'twitter_use',
    'youtube_use',
    'comments',
])

# Pandas can automatically convert JSON into a dataframe.
users = (pd.DataFrame(data, columns=columns)
           .rename(index=str, columns={"id": "user_id"}))

# Export Just Users to Excel
users.to_excel(excel, sheet_name='Users')

## Reformatting user's answers to questions

Now that we have all of the users in their own excel sheet, I need to generate two sheets that contain columns for every question on the application, with one sheet's values indicating whether or not they answered the question correctly, and the other sheet's values indicating the answer they actually gave. This requires a bit of reformatting due to the structure of the JSON response...

The JSON object looks a bit similar to this:
```json
[
    {
        "id": 1,
        "username": "erik",
        "records": [
            {
                "id": 1,
                "correct": true,
                "sentence": [
                    {
                        "text": "wat do u mean"   
                    }
                ]
            }
        ]
    }
]
```

I've only included the data I need in the above JSON example, since this sheet in the document will contain one column for each `record`, with the sentence text as the column header and the correct attribute for the value in that column. In order to do this, I need to accomplish a few things:
    
    1. I need to filter out all of the users that didn't finish each sentence
    2. I need to gather the records for each user in a dataframe 
    3. I need to pivot those records so that the sentence text value is the column header and the values are the correctness
    4. I need to append the pivoted records to the end of each user row so that each row contains each question and the intended value.
    5. I need to reorder the columns

In [3]:
# Generate final DF to be exported to excel
finalCorrect = pd.DataFrame()
finalAnswer = pd.DataFrame()

# Define the necessary columns
recordColumns = ([
    'user_id',
    'answer',
    'correct',
    'sentence.text'
])

# For each user
#   Step 2. Gather the records
#   Step 3. Pivot the dataframe so that the columns become individual sentence text and the user's answers become the row
#   Step 4. Join the pivoted dataframe to the old users dataframe
for user in data:
    # (Step 1) Skip all users that haven't completed every question
    if(user['progress'] != 100):
        continue
        
    # (Step 2) Gather records in wanted format
    records = json_normalize(user['records']) 
    recordsFrame = pd.DataFrame(records, columns=recordColumns)
    
    # Change correct (boolean type) to integer
    recordsFrame['correct'] = recordsFrame['correct'].astype(int)
    
    # (Step 3) Pivot to create a column for each sentence
    recordsCorrectFrame = recordsFrame.pivot(index='user_id', columns='sentence.text', values='correct')
    recordsAnswerFrame = recordsFrame.pivot(index='user_id', columns='sentence.text', values='answer')
    
    # (Step 4) Append recordsFrame to the final product
    finalCorrect = finalCorrect.append(users.merge(recordsCorrectFrame, on='user_id'), sort=True)
    finalAnswer = finalAnswer.append(users.merge(recordsAnswerFrame, on='user_id'), sort=True)


# (Step 5) Reorder columns
updatedColumns = list(users.columns.values)
updatedColumns.extend(list(recordsAnswerFrame.columns.values))

finalCorrect = finalCorrect.reindex(columns=updatedColumns)
finalAnswer = finalAnswer.reindex(columns=updatedColumns)

# Export to excel
finalCorrect.to_excel(excel, sheet_name='Sentence_Correct')
finalAnswer.to_excel(excel, sheet_name='Sentence_Answer')

## Generate averages based on categories

Each of the questions are in one of nine categories:

| Style/Emotion | Positive | Neutral | Negative |
| --- | --- | --- | --- |
| **Abbreviation** | `Abbr_Pos` | `Abbr_Neu` | `Abbr_Neg` |
| **Grammatical** | `Gramm_Pos` | `Gramm_Neu` | `Gramm_Neg` |
| **Emoji** | `Emoji_Pos` | `Emoji_Neu` | `Emoji_Neg` |

In order to determine whether or not our hypothesis is supported, we need to know the average number of sentences the participants guessed correctly for each category. To do that, I went back to the original DataFrame and did the following steps:

1. I gathered all of the records and the necessary column names for each user
2. I generated all 9 averages for each user and appended those to the averages dataframe
    - To do this, I queried all of the user records based on style and emotion for each combination 
3. I filtered to only include people who completed the study

In [4]:
recordColumns = ([
    'correct',
    'sentence.style.id',
    'sentence.style.name',
    'sentence.emotion.id',
    'sentence.emotion.name'
])

##
# Gather means for record scores
# - Used as a lookup table
newColumns = ({
    'Abbr_Pos': ["Abbreviation", "Positive"],
    'Abbr_Neu': ["Abbreviation", "Neutral"], 
    'Abbr_Neg': ["Abbreviation", "Negative"],

    'Gramm_Pos': ["Grammatical", "Positive"],
    'Gramm_Neu': ["Grammatical", "Neutral"], 
    'Gramm_Neg': ["Grammatical", "Negative"],

    'Emoji_Pos': ["Emoji", "Positive"],
    'Emoji_Neu': ["Emoji", "Neutral"], 
    'Emoji_Neg': ["Emoji", "Negative"]
})

averages = pd.DataFrame()

for user in data:
    records = json_normalize(user['records']) # Acquire records    
    
    # Create a data frame with needed columns
    recordsFrame = pd.DataFrame(records, columns=recordColumns)

    # Casting for later where clauses
    recordsFrame['sentence.style.name'] = recordsFrame['sentence.style.name'].astype(str)
    recordsFrame['sentence.emotion.name'] = recordsFrame['sentence.emotion.name'].astype(str)
    
    dic = {'user_id': user['id']}
    for newCol in newColumns:
        # Specify conditions with which to filter the records
        style = newColumns[newCol][0]
        emotion = newColumns[newCol][1]
        conditions = (
            # where sentence.style.name="Style"
            (recordsFrame['sentence.style.name']==style) 
            
            & # AND
        
            # where sentence.emotion.name="Emotion"
            (recordsFrame['sentence.emotion.name']==emotion)
        )
        
        # Filter based on conditions above
        temp = recordsFrame[conditions]
        
        # Obtain the mean of the two selections
        mean = temp['correct'].mean()
        
        # Add that mean to the dictionary of means containing users and 
        dic.update({newCol: mean})
    
    # Once the dictionary contains all 9 averages, append it to the dataframe
    averages = averages.append(users.merge(pd.DataFrame(dic, index=[0]), on='user_id'))
    
# Remove items where participant did not finish each question
finalAverages = averages[averages.progress==100]

# Export
finalAverages.to_excel(excel, sheet_name='Category_Averages')
excel.save()

# Success
print("Successfully exported excel file.")

Successfully exported excel file.
